In [1]:
# Name: Ben Stepaniak
# Section: 100-105
# Data Viz Project: "Climatology"
# Description: This program generates graphs for climate data for a user-selected year.

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import statistics
import numpy as np

# You may add more imports as needed

class WD:
    long = 0
    lat = 1
    jan = 2
    feb = 3
    mar = 4
    apr = 5
    may = 6
    jun = 7
    jul = 8
    aug = 9
    sep = 10
    oct = 11
    nov = 12
    dec = 13
    annual_min = 14
    annual_max = 15
    annual_avg = 16
    annual_sum = 17
    annual_range = 18

In [3]:
def verify_climatology_data(temp_data, precip_data):
    """
    Verifies the structure and format of the climatology data:
        checks that all rows have the same number of columns
        checks that each column is the correct type (float)
        checks that it has the correct number of rows and the correct number of columns
    
    If your data is in the right format, you'll see the output:
        Rows: 85794
        Cols: 18
        Congrats! You have the correct shape of data!
        Rows: 85794
        Cols: 18
        Congrats! You have the correct shape of data!
 
    If you see something else, this will give you an indication of what is not right.
    
    Parameters:
    -----------
    temp_data : list of lists
        From the air_temp data (each sublist corresponds to a row from the input file)
    precip_data : list of lists
        From the precip data (each sublist corresponds to a row from the input file)
    
    """
    data_ind = 0
    data_together = [temp_data, precip_data]
    while data_ind < len(data_together):
        data = data_together[data_ind]
        data_ind += 1

        rows = len(data)
        cols = len(data[0])
        
        # make sure all rows have the same number of columns
        same_number_columns = True
        i = 0
        while i < len(data):
            row = data[i]
            if len(row) != cols:
                same_number_columns = False
            i += 1
                
        if not same_number_columns:
            print("Some rows have a different number of columns!")

        # make sure each column is the correct type
        overall_type = True
        column = 0
        while column < cols:
            col_vals = get_column_values(data, column)
            correct_type = True
            j = 0
            while j < len(col_vals):
                val = col_vals[j]
                if type(val) is not float:
                    correct_type = False
                j += 1
            
            if not correct_type:
                print("Column " + str(column) + " contains values that are not the right type!")
                print(col_vals[:10]) # print the first 10 values
                overall_type = False
            column += 1
        
        
        if overall_type and same_number_columns:
            print("Rows: " + str(rows))
            print("Cols: " + str(cols))
            
            if rows == 85794 and cols == 18:
                print("Congrats! You have the correct shape of data!")
            else:
                print("You have the incorrect shape of data!")
                print("You have # rows (should be 85794): " + str(rows))
                print("You have # columns (should be 18): " + str(cols))

Climatology Data Visualization Project
=============

Add more cells as needed! Make sure that your notebook runs without errors before turning it in for each checkpoint!

In [4]:
def ask_year():
    """
    This function asks the user for a year to analyze.
    The checking and typcasting of the user's input will be handled in the main() function.
    
    Parameters:
    -----------
        None; it takes user input.
            
    Returns:
    -----------
        Whatever the user inputs.
    
    """
    print("Enter a year to analyze (1900-2014) or type 'Exit' to close the program.") # Displays the instructions
    year = input("Please input your selection here: ") # Prompts the user for their input
    return year

In [5]:
def get_menu_choice():
    """
    This function displays a menu of choices and asks the user to input their choice.
    
    Parameters:
    -----------
        None; it takes user input.
            
    Returns:
    -----------
        Whatever menu choice the user selects.
    
    """
    print("You may generate the following graphs:") # Displays the choices
    print()
    print("[1] - Line graphs of global mean precipitation and temperature over 12 months.")
    print("[2] - Histograms of global max/min temperatures and precipitations over 12 months.")
    print("[3] - Line graphs of temperature and precipitation over a month in percentiles (max, 99%, and 50%).")
    print("[4] - Heat maps of mean annual temperature/precipitation.")
    print("[5] - Boxplots for quarterly temperature/precipitation.")
    print("[6] - Pie charts for annual ranges in temperature/precipitation.")
    print("[0] - Exit.")
    print()
    menu_selection = input("Please input your choice here: ") # Prompts the user for their input
    return menu_selection

In [6]:
def read_data(filename):
    """
    This function reads through the specified file and splits each line into a list.
    That list's values are then converted into floats.
    It also calculates and appends each row's max, min, average, and sum to the list.
    It then appends each row to a nested list, which is returned.
    
    Parameters:
    -----------
        filename : The file that will be manipulated.
            
    Returns:
    -----------
        nested_list : A nested list containing each row's values as floats plus the 4 calculations.
    
    """
    nested_list = [] # This is the return list
    file = open(filename, "r") # This opens the file for reading
    line = file.readline() # This initializes the line variable
    while line: # This loop continues as long as there is a line to read
        line_list = line.split() # This strips each row of their white spaces and turns it into a list
        idx = 0 # This is the initialization for the float conversion while loop
        while idx < len(line_list):
            line_list[idx] = float(line_list[idx]) # This replaces each value with its float version
            idx += 1 # This updates the float conversion index variable
        
        current_max = -10000 # This chunk sets up the maximums and minumums for comparison
        current_min = 10000
        
        current_sum = 0 # This initializes the sum of each row
        
        idx_max = 2 # This index variable starts at the January column
        while idx_max <= 13:
            if line_list[idx_max] > current_max: # This checks if the current value is greater than the current max
                current_max = line_list[idx_max] # If it is, the current max is updated
                idx_max += 1 # These 2 lines update the index variable regardless of if the value was a max
            idx_max += 1
        line_list.append(current_max) # The maximum is then appended to the row
        
        idx_min = 2 # This chunk operates using the same principles as the current_maximum chunk
        while idx_min <=13:
            if line_list[idx_min] < current_min:
                current_min = line_list[idx_min]
                idx_min += 1
            idx_min += 1
        line_list.append(current_min) # The minumum is then appended to the row
        
        idx_sum = 2 # Since the average uses the sum in its calculation, the sum is calculated first
        while idx_sum <= 13:
            current_sum += line_list[idx_sum] # Each data point up through December is summed
            idx_sum += 1
        
        avg = current_sum / 12 # The average is then calculated
        
        annual_range = abs(current_max - current_min)
        
        line_list.append(avg) # The average is appended before the sum so that the data fits the WeatherData type
        line_list.append(current_sum) # The sum is appended last
        
        line_list.append(annual_range)
        
        nested_list.append(line_list) # The completed row is appended to the return list
        
        line = file.readline() # This updates the line reading variable
        
    file.close() # This closes the processed file
    return nested_list

In [7]:
def get_located_between(data, column, bound_min, bound_max):
    """
    This utility function returns a list of all values in that fit within the specified bounds in a given column.
    
    Parameters:
    -----------
        data : A nested list of data.
        column : The column of interest.
        bound_min : The minimum value of the bounds, inclusive.
        bound_max : The maximum value of the bounds, non inclusive.
            
    Returns:
    -----------
        bound_list : A list of each data point that fits within the bounds at the specified column.
    
    """
    bound_list = [] # This initializes the return list
    idx = 0 # This initializes the row index
    while idx < len(data): # This loop will continue for each row in the data's nested list
        if (data[idx][column] >= bound_min) and (data[idx][column] < bound_max): # This checks if the data point fits within the bounds
            bound_list.append(data[idx][column]) # If so, it will append the value to the return list
            idx += 1 # Updates the row index
        idx += 1 # Updates the row index
    return bound_list

In [8]:
def get_column_values(data, column):
    """
    This utility function returns a list of all values in the specified column.
    
    Parameters:
    -----------
        data : A nested list of data.
        column : The column of interest.
            
    Returns:
    -----------
        column_list : A list of each data point at the specified column.
    
    """
    column_list = [] # This initializes the return list
    idx = 0 # This initializes the row index
    while idx < len(data): # This loop will continue for each row in the data's nested list
        column_list.append(data[idx][column]) # This appends the column's value to the reurn list
        idx += 1 # Updates the row index
    return column_list

In [9]:
def get_month_names():
    """
    This utility function returns a list of string values that correspond to the months.
    
    Parameters:
    -----------
        None.
            
    Returns:
    -----------
        month_list : A list containing each month as a string value.
    
    """
    month_list = [] # This initializes the return list
    month_list.append("Jan") # This chunk appends each month's abbreviation to the return list
    month_list.append("Feb")
    month_list.append("Mar")
    month_list.append("Apr")
    month_list.append("May")
    month_list.append("Jun")
    month_list.append("Jul")
    month_list.append("Aug")
    month_list.append("Sep")
    month_list.append("Oct")
    month_list.append("Nov")
    month_list.append("Dec")
    return month_list

In [10]:
def generate_line_graph(year, data_type, data):
    """
    This function preps data for line graphing and develops the graph based on the prepped data.
    
    Parameters:
    -----------
        year : The user-defined year, which is used in labeling.
        data_type : The type of data, which is used in labeling.
        data : The actual data which will be used for the graphs.
            
    Returns:
    -----------
        A line graph that shows monthly mean values for precipitation or temperature for a given year.
    
    """
    x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] # This sets up the x ticks to be the months of the year
    x_ticks = get_month_names()
    
    above_cancer = [] # This initializes the data lists for each line
    between_tropics = []
    below_capricorn = []
    idx = 0 # Initializes the index for partitioning the data
    while idx < len(data):
        if data[idx][WD.lat] > 23.43658: # This adds each instance with a latitude above the Tropic of Cancer
            above_cancer.append(data[idx])
            idx += 1
        elif (-23.43658 <= data[idx][WD.lat] <= 23.43658): # This adds each instance with a latitude between the Tropics
            between_tropics.append(data[idx])
            idx += 1
        elif data[idx][WD.lat] < -23.43658: # This adds each insance with a latitude below the Tropic of Cancer
            below_capricorn.append(data[idx])
            idx += 1
        
    above_cancer_avg = [] # This initializes the monthly average list
    month_idx = 2 # This intializes the month index
    while month_idx <= 13:
        month_sum = 0 # Initializes the monthly sum to be used for calculating average
        data_idx = 0 # Initializes a separate index for each line of data
        while data_idx < len(above_cancer): # This loop sums the data for each month
            month_sum += above_cancer[data_idx][month_idx]
            data_idx += 1
        month_avg = month_sum / len(above_cancer) # This calculates the average for each month
        above_cancer_avg.append(month_avg) # This appends each monthly average to the list
        month_idx += 1 # Updates the month
        
    between_tropics_avg = [] # Same principle as the above chunk
    month_idx2 = 2 # While the indexes from the above could be reused, I defined new index variables to avoid confusion
    while month_idx2 <= 13:
        month_sum2 = 0
        data_idx2 = 0
        while data_idx2 < len(between_tropics):
            month_sum2 += between_tropics[data_idx2][month_idx2]
            data_idx2 += 1
        month_avg2 = month_sum2 / len(between_tropics)
        between_tropics_avg.append(month_avg2)
        month_idx2 += 1
        
    below_capricorn_avg = [] # Same principle as the above chunk
    month_idx3 = 2
    while month_idx3 <= 13:
        month_sum3 = 0
        data_idx3 = 0
        while data_idx3 < len(below_capricorn):
            month_sum3 += below_capricorn[data_idx3][month_idx3]
            data_idx3 += 1
        month_avg3 = month_sum3 / len(below_capricorn)
        below_capricorn_avg.append(month_avg3)
        month_idx3 += 1
    
    plt.xticks(x, x_ticks) # This is where the actual graphing takes place
    plt.plot(x, above_cancer_avg, label = "Above Tropic of Cancer")
    plt.plot(x, between_tropics_avg, label = "Between Tropics")
    plt.plot(x, below_capricorn_avg, label = "Below Tropic of Capricorn")
    
    if data_type == "temp": # This conditional runs if the data is temperature
        plt.title("Global mean temperature values over 12 months for the year " + year)
        plt.ylabel("Temperature (degrees celsius)")
        plt.xlabel("Month")
        plt.legend(bbox_to_anchor = (1, 1), loc = "upper left") # The kwargs here ensure the legend is outside of the graph
        plt.savefig("monthly_avg_temp_line.png", dpi = 275, bbox_inches = "tight") # The kwargs here set the size of the saved image
        plt.show()
        
    elif data_type == "precip": # This conditional runs if the data is precipitation
        plt.title("Global mean precipitation values over 12 months for the year " + year)
        plt.ylabel("Precipitation (mm)")
        plt.xlabel("Month")
        plt.legend(bbox_to_anchor = (1, 1), loc = "upper left")
        plt.savefig("monthly_avg_precip_line.png", dpi = 275, bbox_inches = "tight")
        plt.show()

In [11]:
def generate_histogrms(user_bins, year, data_type, data):
    """
    This function preps data for a histogram and develops a histogram based upon the prepped data.
    
    Parameters:
    -----------
        user_bins : The amount of bins selected by the user, used in graphing.
        year : The user-defined year, which is used in labeling.
        data_type : The type of data, which is used in labeling.
        data : The actual data which will be used for the graphs.
            
    Returns:
    -----------
        A histogram that shows minimum and maximum values for a given year.
    
    """
    min_data = get_column_values(data, WD.annual_min) # These 2 lines prep the minimum and maximum data values
    max_data = get_column_values(data, WD.annual_max)
    
    plt.hist([min_data, max_data], bins = user_bins, histtype = "bar", stacked = False, label = ["Maximums", "Minimums"]) # This line produces the actual histogram
    
    if data_type == "temp": # These conditionals execute based on the data_type parameter to produce the appropriate labels
        plt.title("Histogram of min/max global temperature for the year " + year)
        plt.ylabel("Number of locations")
        plt.xlabel("Temperature (degrees celcius)")
        plt.legend() # I left out kwargs for this one as to match the example graphs more closely
        plt.savefig("min_max_temp_hist.png", dpi = 275, bbox_inches = "tight")
        plt.show()
        
    elif data_type == "precip":
        plt.title("Histogram of min/max global precipitation for the year " + year)
        plt.ylabel("Number of locations")
        plt.xlabel("Precipitation (mm)")
        plt.legend()
        plt.savefig("min_max_precip_hist.png", dpi = 275, bbox_inches = "tight")
        plt.show()

In [12]:
def generate_percentile_graphs(year, data_type, data):
    """
    This function preps data for a line graph that shows monthly maximums and percentiles.
    
    Parameters:
    -----------
        year : The user-defined year, which is used in labeling.
        data_type : The type of data, which is used in labeling.
        data : The actual data which will be used for the graphs.
            
    Returns:
    -----------
        A line graph that shows the maximum, 99th percentile, and 50th percentile of monthly temperature or precipitation for a given year.
    
    """
    x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] # This sets up the x ticks to be the months of the year
    x_ticks = get_month_names()
    
    list_max = [] # These initialize the lists for the max, 99th, and 50th percentile
    list_99 = []
    list_50 = []
    month_idx = 2 # Initializes the monthly counter index
    while month_idx <= 13:
        month_list = get_column_values(data, month_idx) # Gets the values for the current month
        month_max = max(month_list) # Finds the maximum
        list_max.append(month_max) # Appends it to a list
        month_99 = np.percentile(month_list, 99) # Finds the 99th percentile
        list_99.append(month_99) # Appends it to a list
        month_50 = np.percentile(month_list, 50) # Finds the 50th percentile
        list_50.append(month_50) # Appends it to a list
        month_idx += 1 # Update the month index to go to the next month
    
    plt.xticks(x, x_ticks) # This is where the actual graphing takes place
    plt.plot(x, list_max, label = "Maximum")
    plt.plot(x, list_99, label = "99th Percentile")
    plt.plot(x, list_50, label = "50th Percentile")
    
    if data_type == "temp": # Labeling path uses same principles as the first line graph
        plt.title("Temperature observations per month in year " + year)
        plt.ylabel("Temperature (degrees celsius)")
        plt.xlabel("Month")
        plt.legend(bbox_to_anchor = (1, 1), loc = "upper left") # The kwargs here ensure the legend is outside of the graph
        plt.savefig("percentile_temp_line.png", dpi = 275, bbox_inches = "tight") # The kwargs here set the size of the saved image
        plt.show()
        
    elif data_type == "precip":
        plt.title("Precipitation observations per month in year " + year)
        plt.ylabel("Precipitation (mm)")
        plt.xlabel("Month")
        plt.legend(bbox_to_anchor = (1, 1), loc = "upper left") # The kwargs here ensure the legend is outside of the graph
        plt.savefig("percentile_precip_line.png", dpi = 275, bbox_inches = "tight") # The kwargs here set the size of the saved image
        plt.show()

In [13]:
def generate_heatmap(year, data_type, data):
    """
    This function generates a heatmap scatterplot for temperature or precipitation (on a log scale) for a given year.
    
    Parameters:
    -----------
        year : The user-defined year, which is used in labeling.
        data_type : The type of data, which is used in labeling.
        data : The actual data which will be used for the graphs.
            
    Returns:
    -----------
        A heatmap scatterplot for temperature or precipitation for a given year.
    
    """
    longitude = get_column_values(data, WD.long) # These two lines prep the x and y-axises
    latitude = get_column_values(data, WD.lat)
    
    if data_type == "temp":
        temp_values = get_column_values(data, WD.annual_avg) # This gathers the annual average values
        plt.title("Heat map of temperature values in year " + year)
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.scatter(longitude, latitude, c = temp_values) # This does the actual plotting
        plt.colorbar()
        plt.savefig("temp_heatmap.png", dpi = 275, bbox_inches = "tight")
        plt.show()
        
    elif data_type == "precip":
        precip_values = get_column_values(data, WD.annual_avg)
        plt.title("Heat map of precipitation values in year " + year)
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.scatter(longitude, latitude, c = precip_values, norm = colors.LogNorm()) # The extra kwarg here ensures that the data will be on a logarithmic scale
        plt.colorbar()
        plt.savefig("precip_heatmap.png", dpi = 275, bbox_inches = "tight")
        plt.show()

In [14]:
def generate_boxplot(year, data_type, data):
    """
     This function generate 4 boxplots for quarterly temperature or precipitation (on a log scale) for a given year.
    
    Parameters:
    -----------
        year : The user-defined year, which is used in labeling.
        data_type : The type of data, which is used in labeling.
        data : The actual data which will be used for the graphs.
            
    Returns:
    -----------
        4 boxplots showing quarterly temperature or precipitation for a given year.
    
    """
    jan = get_column_values(data, WD.jan) # These 4 chunks just gather the data for each quarter
    feb = get_column_values(data, WD.feb)
    mar = get_column_values(data, WD.mar)
    quarter1 = jan + feb + mar
    
    apr = get_column_values(data, WD.apr)
    may = get_column_values(data, WD.may)
    jun = get_column_values(data, WD.jun)
    quarter2 = apr + may + jun
    
    jul = get_column_values(data, WD.jul)
    aug = get_column_values(data, WD.aug)
    sep = get_column_values(data, WD.sep)
    quarter3 = jul + aug + sep
    
    octo = get_column_values(data, WD.oct)
    nov = get_column_values(data, WD.nov)
    dec = get_column_values(data, WD.dec)
    quarter4 = octo + nov + dec
    
    fig, axs = plt.subplots(2, 2) # This sets up and plots the 4 quarterly boxplots
    axs[0, 0].boxplot(quarter1)
    axs[1, 0].boxplot(quarter2)
    axs[0, 1].boxplot(quarter3)
    axs[1, 1].boxplot(quarter4)
    fig.subplots_adjust(left=0.08, right=0.98, bottom=0.05, top=0.9, hspace=0.9, wspace=0.3)
    
    if data_type == "temp": # Labeling chunk
        axs[0, 0].set_title("Quarter 1 box plot of\ntemperature in year " + year)
        axs[0, 0].set_ylabel("Temp (degrees celcius)")
        axs[1, 0].set_title("Quarter 2 box plot of\ntemperature in year " + year)
        axs[1, 0].set_ylabel("Temp (degrees celcius)")
        axs[0, 1].set_title("Quarter 3 box plot of\ntemperature in year " + year)
        axs[0, 1].set_ylabel("Temp (degrees celcius)")
        axs[1, 1].set_title("Quarter 4 box plot of\ntemperature in year " + year)
        axs[1, 1].set_ylabel("Temp (degrees celcius)")
        plt.savefig("quartly_temp_boxplot.png", dpi = 275, bbox_inches = "tight")
        
    elif data_type == "precip":
        axs[0, 0].set_ylabel("Precipitation (mm)")
        axs[0, 0].set_yscale("log") # This ensures that the precipitation scale will be logarithmic
        axs[0, 0].set_title("Quarter 1 box plot of\nprecipitation in year " + year)
        axs[1, 0].set_ylabel("Precipitation (mm)")
        axs[1, 0].set_yscale("log")
        axs[1, 0].set_title("Quarter 2 box plot of\nprecipitation in year " + year)
        axs[0, 1].set_ylabel("Precipitation (mm)")
        axs[0, 1].set_yscale("log")
        axs[0, 1].set_title("Quarter 3 box plot of\nprecipitation in year " + year)
        axs[1, 1].set_ylabel("Precipitation (mm)")
        axs[1, 1].set_yscale("log")
        axs[1, 1].set_title("Quarter 4 box plot of\nprecipitation in year " + year)
        plt.savefig("quartly_precip_boxplot.png", dpi = 275, bbox_inches = "tight")

In [15]:
def generate_pie_chart(year, data_type, data):
    """
     This function generates a pie chart for temperature or precipitation ranges for a given year.
    
    Parameters:
    -----------
        year : The user-defined year, which is used in labeling.
        data_type : The type of data, which is used in labeling.
        data : The actual data which will be used for the graphs.
            
    Returns:
    -----------
        A pie chart for temperature or precipitation ranges for a given year.
    
    """
    if data_type == "temp": # This divides the data up into temperature ranges
        below10 = []
        inc10_to20 = []
        inc20_to30 = []
        inc30_to40 = []
        inc40_above = []
        idx1 = 0 # Initializes the data index
        while idx1 < len(data):
            if data[idx1][WD.annual_range] < 10: # Each comparison checks if the current range fits and places them in their appropriate list
                below10.append(data[idx1][WD.annual_range])
                idx1 += 1
            elif 10 <= data[idx1][WD.annual_range] < 20:
                inc10_to20.append(data[idx1][WD.annual_range])
                idx1 += 1
            elif 20 <= data[idx1][WD.annual_range] < 30:
                inc20_to30.append(data[idx1][WD.annual_range])
                idx1 += 1
            elif 30 <= data[idx1][WD.annual_range] < 40:
                inc30_to40.append(data[idx1][WD.annual_range])
                idx1 += 1
            elif 40 <= data[idx1][WD.annual_range]:
                inc40_above.append(data[idx1][WD.annual_range])
                idx1 += 1
                
        labels = "Below 10", "Between 10 and 20", "Between 20 and 30", "Between 30 and 40", "Above 40" # Labeling each range
        sizes = [len(below10), len(inc10_to20), len(inc20_to30), len(inc30_to40), len(inc40_above)] # This takes the length of each list to determine the appropriate portion
        
        fig, axs = plt.subplots() # This chunk labels and generates the chart
        axs.pie(sizes, labels = labels, autopct = "%1.1f%%") 
        axs.axis("equal")
        fig.set_facecolor("w")
        axs.set_title("Ranges of temperature for the year " + year + "\n(degrees celcius)", pad = 20)
        plt.show()
        fig.savefig("temp_ranges_pie.png", dpi = 275, bbox_inches = "tight")
        
    elif data_type == "precip": # Same principle as the temperature chunk; there is no variation to the structure of the code, only different ranges
        below20 = []
        inc20_to40 = []
        inc40_to60 = []
        inc60_to80 = []
        inc80_to100 = []
        inc100_above = []
        idx2 = 0
        while idx2 < len(data):
            if data[idx2][WD.annual_range] < 20:
                below20.append(data[idx2][WD.annual_range])
                idx2 += 1
            elif 20 <= data[idx2][WD.annual_range] < 40:
                inc20_to40.append(data[idx2][WD.annual_range])
                idx2 += 1
            elif 40 <= data[idx2][WD.annual_range] < 60:
                inc40_to60.append(data[idx2][WD.annual_range])
                idx2 += 1
            elif 60 <= data[idx2][WD.annual_range] < 80:
                inc60_to80.append(data[idx2][WD.annual_range])
                idx2 += 1
            elif 80 <= data[idx2][WD.annual_range] < 100:
                inc80_to100.append(data[idx2][WD.annual_range])
                idx2 += 1
            elif 100 <= data[idx2][WD.annual_range]:
                inc100_above.append(data[idx2][WD.annual_range])
                idx2 += 1
        
        labels = "Below 20", "Between 20 and 40", "Between 40 and 60", "Between 60 and 80", "Between 80 and 100", "Above 100"
        sizes = [len(below20), len(inc20_to40), len(inc40_to60), len(inc60_to80), len(inc80_to100), len(inc100_above)]
        
        fig, axs = plt.subplots()
        axs.pie(sizes, labels = labels, autopct = "%1.1f%%")
        axs.axis("equal")
        fig.set_facecolor("w")
        axs.set_title("Ranges of precipitation for the year " + year + "\n(mm)", pad = 20)
        plt.show()
        fig.savefig("precip_ranges_pie.png", dpi = 275, bbox_inches = "tight")

In [16]:
def main():
    """
    The main() function, which calls previously defined functions in order to determine the year of data to process
    and generate graphs based upon user selection.
    
    Parameters:
    -----------
        None.
            
    Returns:
    -----------
        Whatever graphs the user wishes to generate of a given year.
    
    """
    user_year = ask_year() # This line calls the ask_year() function and saves the input as a variable
    
    if (not user_year.isdecimal()) or (not 1900 <= int(user_year) <= 2014): # This checks if the user has entered a valid input
        if user_year.upper() == "EXIT": # If they specifically entered "Exit", then an exit message is displayed before termination
            print()
            print("You have exited the program.")
            print("Terminating program...")
            print()
            print("----------------------------")
            print()
            print("Program closed.")
            return
        print()
        print("You have not entered a valid choice.") # If the user inputted anything other than a year in 1900-2014 or "Exit", an error message is displayed before termination
        print("Terminating program...")
        print()
        print("------------------------------------")
        print()
        print("Program closed.")
        return
    
    print()
    print("You have selected the year", user_year) # This message confirms what year the user selected
    print()
    print("Processing the year's precipitation and air temperature data...")
    print()
    print("---------------------------------------------------------------")
    print()
    
    
    temp_path = "data/air_temp_2014/air_temp." + user_year # This chunk defines the file path based on the user's input
    precip_path = "data/precip_2014/precip." + user_year
    
    temp_data = read_data(temp_path) # This chunk calls the read_data() function and saves the resulting nested lists as variables
    precip_data = read_data(precip_path)
    
    user_choice = get_menu_choice() # This calls the get_menu_choice() function and stores the user's selection as a variable for later use
    
    while (not user_choice.isdecimal()) or (not 0 <= int(user_choice) <= 6): # If the user made an invalid or out of bounds selection, an error is displayed and the menu is called again
        print()
        print("You have not entered a valid menu choice.  Redisplaying menu...")
        print()
        print("---------------------------------------------------------------")
        user_choice = get_menu_choice()
        
    if int(user_choice) == 0: # This chunk just individually defines each menu choice
        print()
        print("You have exited the program.")
        print("Terminating program...")
        print()
        print("----------------------------")
        print()
        print("Program closed.")
        return
    elif int(user_choice) == 1: # This governs the line graph choice
        print()
        print("You have selected: Line graphs of global mean precipitation and temperature over 12 months.")
        print("Generating graphs...")
        print()
        print("-------------------------------------------------------------------------------------------")
        print()
        generate_line_graph(user_year, "temp", temp_data) # These call the line graph-generating function for each graph
        generate_line_graph(user_year, "precip", precip_data)
        print()
        print("-------------------------------------------------------------------------------------------")
        print()
        print("Your graphs have been saved as .png images:")
        print("[1] monthly_avg_temp_line.png")
        print("[2] monthly_avg_precip_line.png")
    elif int(user_choice) == 2: # This governs the histogram choice
        print()
        print("You have selected: Histograms of global max/min temperatures and precipitations over 12 months.")
        print()
        bin_size = input("How many bins would you like?: ") # This asks the user for their preferred bin size
        while not bin_size.isdecimal(): # This checks the validity of the user selection
            print()
            print("You have not selected a valid bin size.")
            print()
            bin_size = input("How many bins would you like?: ")
        bin_size = int(bin_size) # Typecasts the bin size as an integer
        print()
        print("Generating graphs...")
        print()
        print("-----------------------------------------------------------------------------------------------")
        print()
        generate_histogrms(bin_size, user_year, "temp", temp_data) # These lines call the histogram-generating function for each graph
        generate_histogrms(bin_size, user_year, "precip", precip_data)
        print()
        print("-----------------------------------------------------------------------------------------------")
        print()
        print("Your graphs have been saved as .png images:")
        print("min_max_temp_hist.png")
        print("min_max_precip_hist.png")
    elif int(user_choice) == 3:
        print()
        print("You have selected: Line graphs of temperature and precipitation over a month in percentiles (max, 99%, and 50%).")
        print("Generating graphs...")
        print()
        print("----------------------------------------------------------------------------------------------------------------")
        print()
        generate_percentile_graphs(user_year, "temp", temp_data) # These lines call the precentile line graph-generating function for each graph
        generate_percentile_graphs(user_year, "precip", precip_data)
        print()
        print("----------------------------------------------------------------------------------------------------------------")
        print()
        print("Your graphs have been saved as .png images:")
        print("percentile_temp_line.png")
        print("percentile_precip_line.png")
    elif int(user_choice) == 4:
        print()
        print("You have selected: Heat maps of mean annual temperature/precipitation.")
        print("Generating graphs...")
        print()
        print("----------------------------------------------------------------------")
        print()
        generate_heatmap(user_year, "temp", temp_data) # These lines call the heatmap-generating function for each graph
        generate_heatmap(user_year, "precip", precip_data)
        print()
        print("----------------------------------------------------------------------")
        print()
        print("Your graphs have been saved as .png images:")
        print("temp_heatmap.png")
        print("precip_heatmap.png")
    elif int(user_choice) == 5:
        print()
        print("You have selected: Boxplots for quarterly temperature/precipitation.")
        print("Generating graphs...")
        print()
        print("--------------------------------------------------------------------")
        print()
        generate_boxplot(user_year, "temp", temp_data) # These lines call the boxplot-generating function for each graph
        generate_boxplot(user_year, "precip", precip_data)
        print("Your graphs have been saved as .png images:")
        print("quartly_temp_boxplot.png")
        print("quartly_precip_boxplot.png")
        print()
    elif int(user_choice) == 6:
        print()
        print("You have selected: Pie charts for annual ranges in temperature/precipitation.")
        print("Generating graphs...")
        print()
        print("-----------------------------------------------------------------------------")
        print()
        generate_pie_chart(user_year, "temp", temp_data) # These lines call the pie chart-generating function for each graph
        generate_pie_chart(user_year, "precip", precip_data)
        print()
        print("-----------------------------------------------------------------------------")
        print()
        print("Your graphs have been saved as .png images:")
        print("temp_ranges_pie.png")
        print("precip_ranges_pie.png")

In [17]:
main()

Enter a year to analyze (1900-2014) or type 'Exit' to close the program.


Please input your selection here:  exit



You have exited the program.
Terminating program...

----------------------------

Program closed.
